# Which Python string formatting method should you be using in your data science project?

**3rd July 2021**

[Engineering for Data Science post](https://engineeringfordatascience.com/posts/python_string_formatting_for_data_science/)

## Python String Formatting

String formatting (also known as string interpolation) is the process of inserting a custom string or variable into a predefined 'template' string.

In Python, there are four methods for formatting strings (also known as string interpolation):
- % operator
- format
- f-strings
- Templates

This is a little confusing, even by Python's own manifesto, the Zen of Python:

> [Zen of Python](https://www.python.org/dev/peps/pep-0020/) - "There should be one-- and preferably only one --obvious way to do it."

So why are there four native methods for formatting strings in Python?

Each has their own trade-offs and benefits of simplicity, flexibility, and/or extensibility. But what are the differences, which one should you use for which purpose and why?

In the context of data science there are two common use cases for formatting strings:
1. Print statements
2. SQL queries

In this post, we will go through each use case and describe which string formatting method might be most appropriate.

## 1. Print Statements

String interpolation in data science is particularly useful for logging (e.g. during model training); creating dynamic chart titles and printing statistics.

In Python, the three most common methods for this purpose are `% operator`, `format`  or `f-strings`. Additionally, there is the `Template` method, however, this is seldom used.

Let's briefly discuss each of these methods.

### % operator - 'Old method' 

We will start with the `%` operator method to get it out of the way.

String placeholders are denoted by a `%` symbol, followed by a character or characters which specify the desired formatting.

**Example:**

In [1]:
rows = 10
columns = 4

# print string representation
print("My data has %s rows and %s columns" % (rows, columns))

My data has 10 rows and 4 columns


It is also possible to use named placeholders and supply a dictionary which can make the statement more readable.

In [2]:
data = {"rows": rows, "columns": columns}

# print with named placehodlers
print("My data has %(rows)s rows and %(columns)s columns" % data)

My data has 10 rows and 4 columns


The `%` operator method is generally seen as a legacy method for string interpolation and should not be used in favour of the `format` or `f-string` methods described next.

Common grievances with this method include:
- The `%` notation can be hard to read
- `%` notation can be confused with the modulus operator. 
- The syntax can also lead to common errors such as [failing to display tuples and dictionaries correctly](https://docs.python.org/3/library/stdtypes.html?highlight=sprintf#printf-style-string-formatting).

Unless you are using a Python version less than 2.7, you should avoid using this method.

So, let's quickly move on...

### str.format() - 'Newer method'

Since Python 3 (and now backported to Python 2.7), you can format strings by calling the `.format()` method on the string object.

The functionality is very similar to the previous `%` operator formatting, however, the string placeholders are denoted by `{}` which can be more readable.

A full list of formatting functionality is available at [pyformat.info](https://pyformat.info/) which provides a great 'cheat sheet' for all the various ways to format a string (e.g. rounding, date time formatting etc.) - I would highly recommend checking it out.

**Example:**

In [3]:
# print string representation
print("My data has {} rows and {} columns".format(rows, columns))

My data has 10 rows and 4 columns


In [4]:
# print with named placehodlers
print("My data has {rows} rows and {columns} columns".format(**data))

My data has 10 rows and 4 columns


`str.format()` is an improvement on `%`, however, the syntax can be a bit verbose, particularly if you have a lot of variables to substitute.

### f-strings - 'Newest method'

Finally, since Python 3.6, there is a third method called string literals or 'f-strings' which lets you use embedded Python expressions inside string constants.

This can be really useful as it removes some of the verbose syntax overhead of the previous methods which reduces the amount of code you need to write.

With this method you only need to precede the string with the letter `f` or `F`.

**Example**

In [5]:
print(f"My data has {rows} rows and {columns} columns")

My data has 10 rows and 4 columns


### Template

The Python programming language comes with a [standard library](https://docs.python.org/3/tutorial/stdlib2.html#templating) called string which has a useful method called `Template`. 

Perhaps, the least common string formatting method, the Template syntax denotes placeholder using `$` followed by the variable name to replace:

In [6]:
from string import Template

temp = Template("There are $rows rows and $columns in the data")
temp.substitute(rows=rows, columns=columns)

'There are 10 rows and 4 in the data'

Using the Template method is normally overkill and unnecessary for simple print statements, however, it can be particularly useful for [protecting your application from malicious actors](https://realpython.com/python-string-formatting/#4-template-strings-standard-library) if you require user input.

### Which method should you use?

That was a *very* brief intro to the three main methods of string formatting in Python. I recommend checking out [RealPython](https://realpython.com/python-string-formatting/) and [pyformat](https://pyformat.info) for more detailed information on each method and the various different ways to customise the formatting.

Generally, we can exclude the `%` and `Template` methods unless using legacy code bases (e.g. Python <2.7) or dealing with user inputs with the potential for malicious activity.

So that leaves us with `str.format()` or `f-strings`.

**For print statements use f-strings for most use cases.**

The syntax is very easy to remember and is less verbose than the `str.format()` method which makes it easier to read. You can also include expressions within the string which can be useful for making on the fly calculations. For example:


In [7]:
input_list = [1.3, 4.98, 32, 5.32, 3.98, 6.1, 2.4, 10.4]

print(f"The average value of the input list is {sum(input_list)/len(input_list):.2f}")

The average value of the input list is 8.31


However, there are a couple cases where `str.format()` can be more practical. The main example being when you are using a dictionary as the input source for your substitution variables.

For example, if you want to pass a dictionary containing the configuration or metadata for a particular model into a string which logs the training to the console.

Using an f-string, you have to specify the name of the dictionary each time you want to access a key. This involves a lot of repeated typing. It also reduces the flexibility of your statement if you want to pass a dictionary with a different name into the statement. You can also get in a mess with single and double quotes when referencing the dictionary key inside the wider print statement.

In [8]:
metadata = {"model": "xgboost", "model_dir": "models/", "data_dir": "data/"}

# interpolation using f-strings
print(
    f"Training {metadata['model']} model on data in the "
    f"'{metadata['data_dir']}' directory)..."
)

Training xgboost model on data in the 'data/' directory)...


A better and more flexible approach in this scenario would be to use the `str.format()` method and unpack the input dictionary containing the metadata.

In [9]:
print(
    "Training {model} model on data in the '{data_dir}' directory...".format(**metadata)
)

Training xgboost model on data in the 'data/' directory...


## 2. SQL Queries

The second major use case is string interpolation for SQL queries. This is probably the least trivial use case as there can be added complexity, especially if you want to generate long queries dynamically.

There are two general cases where you will be working with SQL queries in Python:
1. 'In-line' in a Notebook or Python script
2. Importing from a .sql file

Both scenarios can be treated in a similar way, because when you import from a `.sql` file you are essentially just reading a string.

It is common deal with dynamic sql querys by developing a 'base' sql query with placeholders and substituting the placeholders with the required values for your particular analysis. 

For example if we wanted to get the daily value of orders for a particular city we might have a base query defined as follows:

In [10]:
base_sql_query = """
SELECT
    date,
    SUM(order_value)
FROM orders
WHERE city = '{city}'
GROUP BY date
"""

We could then apply string formatting using the `str.format()` method and build the query for a particular city dynamically.

The function below takes the base sql query and inserts the specified city into the query.

In [11]:
def build_orders_by_city_query(city: str, base_sql_query: str = base_sql_query) -> str:
    return base_sql_query.format(city=city)


print(build_orders_by_city_query(city="London"))


SELECT
    date,
    SUM(order_value)
FROM orders
WHERE city = 'London'
GROUP BY date



We could make this function even more generalisable to build any query from a input dictionary containing the variables we want to substitute in that query.  We can unpack the variables dictionary to populate the string placeholders using the `str.format()` method.

In [12]:
def build_query(variables: dict, base_sql_query: str = base_sql_query) -> str:
    return base_sql_query.format(**variables)


base_sql_query = """
SELECT
    date,
    SUM(order_value)
FROM orders
WHERE city = '{city}' AND date > '{start_date}'
GROUP BY date
"""

variables = {"city": "London", "start_date": "2020-01-01"}
print(build_query(variables, base_sql_query))


SELECT
    date,
    SUM(order_value)
FROM orders
WHERE city = 'London' AND date > '2020-01-01'
GROUP BY date



Here we have been able to extent the initial base query by adding an additional filter (start_date) to the input dictionary.

Note that we could have used the `f-string` method, however, this is a use case where we are likely to be dealing with dictionary inputs so the `str.format()` is preferable as it easily allows us to easily unpack many variables from a dictionary input.

This works fine for small queries where the structure of the query is 'static' - i.e. you always want to filter by the same columns or always want to apply the same arithmetic operations.

However, what happens if we want to make a longer and more complex query? For example, if, depnding on the situation we want to filter by multiple fields or by no fields at all. Or if we want to dynamically unpivot certain rows depending on their value. 

With the current approach we have to specify a fairly rigid base query ahead of time which is inflexible to any change in the query logic.

Luckily, there is a fifth approach to string interpolation for SQL queries - Jinja templates.

### Jinja

[Jinja](https://jinja.palletsprojects.com/en/3.0.x/intro/#introduction) is a fast, expressive and extensible templating engine which allows us to incorporate simple logic into our string expressions. 

Jinja's main use case is for rendering HTML templates for web applications, however, it comes in handy for building SQL queries as well.

I won't go into the syntax details too much in this post, rather, just demonstrate how it is a very powerful templating engine which allows you to program simple loops and if statements into your strings.

Going back to the previous example we can create the following Jinja template which will generalise to our needs.

In [13]:
jinja_base_sql_query = """
SELECT
    date,
    SUM(order_value)
FROM orders
WHERE
    {%- for city in filter_cities %}
    city = '{{city}}'
    {% if not loop.last -%}
    AND
    {%- endif -%}
    {%- endfor %}
GROUP BY date
"""

This Jinja templated query includes a 'for loop' in the `WHERE` clause which will dynamically create a filter for each city provided in a list called `filter_cities`. 

In [14]:
from jinja2 import Template

filter_cities = ["London", "Cardiff", "Edinburgh"]

print(Template(jinja_base_sql_query).render(filter_cities=filter_cities))


SELECT
    date,
    SUM(order_value)
FROM orders
WHERE
    city = 'London'
    AND
    city = 'Cardiff'
    AND
    city = 'Edinburgh'
    
GROUP BY date


We have improved from the previous examples as we now have the ability to filter by an arbitrary list of cities - imagine if we had to write this query manually with a long list of cities!

We can take this further by applying logic to the columns we want to select as well as the cities we want to filter by.

In [15]:
jinja_base_sql_query2 = """
SELECT
    date
    {%- for product in target_products %}
    , SUM(CASE WHEN product_name = '{{product}}' THEN order_value END) AS sum_{{product}}_value
    {%- endfor %}
FROM orders
{% if cities_filter -%}
WHERE
    {%- for city in cities_filter %}
    city = '{{city}}'
    {% if not loop.last -%}
    AND
    {%- endif -%}
    {%- endfor %}
{% endif -%}
GROUP BY date
"""  # noqa: E501

In [16]:
query_data = {
    "target_products": ["book", "pen", "paper"],
    "cities_filter": ["London", "Cardiff", "Edinburgh"],
}

print(Template(jinja_base_sql_query2).render(query_data))


SELECT
    date
    , SUM(CASE WHEN product_name = 'book' THEN order_value END) AS sum_book_value
    , SUM(CASE WHEN product_name = 'pen' THEN order_value END) AS sum_pen_value
    , SUM(CASE WHEN product_name = 'paper' THEN order_value END) AS sum_paper_value
FROM orders
WHERE
    city = 'London'
    AND
    city = 'Cardiff'
    AND
    city = 'Edinburgh'
    
GROUP BY date


Here we have pivoted the product_name column to get the daily value of three products we are most interested in and also applied some city filters. If we want to change the number of cities to filter or remove them completely, we don't need to make any changes to the base query, only to the input dictionary (query_data).

In [17]:
# removed cities_filter
query_data = {"target_products": ["book", "pen", "paper"]}

print(Template(jinja_base_sql_query2).render(query_data))


SELECT
    date
    , SUM(CASE WHEN product_name = 'book' THEN order_value END) AS sum_book_value
    , SUM(CASE WHEN product_name = 'pen' THEN order_value END) AS sum_pen_value
    , SUM(CASE WHEN product_name = 'paper' THEN order_value END) AS sum_paper_value
FROM orders
GROUP BY date


As you can see, by removing the `cities_filter` key from the `query_data` input dictionary we have completely removed the `WHERE` clause.

These examples are slightly contrived but I hope they demonstrate the power of Jinja templating for your SQL queries to make them more expressive and generalisable.

The great thing about Jinja templates is that they are portable. You could save the Jinja templates as a `.sql` file and they can be reused across multiple projects. An alternative would be to create your own custom Python function to build up the complex query string dynamically, however, you would have to transport that function around with the SQL file. With Jinja, you just need to import `jinja2` and away you go.

This approach is used by SQL pipeline tools such as [dbt](https://www.getdbt.com/).

## Key Takeaways

There are many different ways to format strings in Python. In data science you are most likely going to be requiring string interpolation for creating descriptive print statements or building dynamic SQL queries. Which string formatting method you use will depend on your use case, however, my rule of thumb is:

- Use f-strings as the first choice for print statements
- Resort to `str.format()` if dealing with dictionaries as inputs (useful for building simple SQL queries)
- Use Jinja templating for reproducible and generalisable SQL queries


Summarised by this flowchart.

Happy coding!

## Resources